In [4]:
import ctypes
import numpy as np
import utility.Pi_util
import importlib

In [5]:
importlib.reload(utility.Pi_util)
path = "/media/rpiControl/FF5E-7541/test_cpp/"
test = utility.Pi_util.parse_chunks_binary(path)

Parsing: b'/media/rpiControl/FF5E-7541/test_cpp/chunk_1.bin'
Name: M_size | Val: 3256
Name: W_size | Val: 9600000
Name: P_size | Val: 192000000
Name: S_size | Val: 0


In [ ]:
print(test)